In [8]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [9]:
data = pd.read_csv('20190301.as-rel2.txt',delimiter='\n', comment='#', header=None, encoding='ISO-8859-1')

In [10]:
data['provider_peer'], data['customer_peer'], data['relation_type'], data['source'] = data[0].str.split('|').str

In [11]:
data['relation_type']  = data['relation_type'].apply(lambda x: 'provider-customer' if x == '-1' else 'peer-peer')

In [12]:
links = data[['provider_peer','customer_peer','relation_type']]

In [13]:
ases = set(links['provider_peer'].values)
ases.update(links['customer_peer'].values)

In [14]:
ases

{'22147',
 '7306',
 '47973',
 '37587',
 '13845',
 '39057',
 '54547',
 '22892',
 '51865',
 '25411',
 '42350',
 '207161',
 '11851',
 '14909',
 '203416',
 '54561',
 '33201',
 '26025',
 '8553',
 '59696',
 '48726',
 '12507',
 '28929',
 '14495',
 '6428',
 '39897',
 '32588',
 '264839',
 '41171',
 '18298',
 '267226',
 '8978',
 '264973',
 '197958',
 '25651',
 '262685',
 '2601',
 '14989',
 '38888',
 '28017',
 '264872',
 '61438',
 '32551',
 '18784',
 '45460',
 '23284',
 '7837',
 '34832',
 '50663',
 '52332',
 '35616',
 '57205',
 '201315',
 '7045',
 '328137',
 '35798',
 '15117',
 '38326',
 '133699',
 '205212',
 '28676',
 '10200',
 '8339',
 '394733',
 '35778',
 '134441',
 '30496',
 '328202',
 '135806',
 '9797',
 '199842',
 '14735',
 '49890',
 '21293',
 '394878',
 '63036',
 '38406',
 '55188',
 '11452',
 '61004',
 '46259',
 '2043',
 '53886',
 '42065',
 '133193',
 '52246',
 '327706',
 '4040',
 '26130',
 '63741',
 '136275',
 '202784',
 '136305',
 '14587',
 '202723',
 '52986',
 '27221',
 '31850',
 '19399